## Notebook Purpose
- Build weekly user snapshot dataset with fixed windows (history=23d, label=7d)
- Create user-level features (activity counts, ratios, recency, price stats)
- Assign time-based splits (train/valid/test) using label window boundaries
- Save dataset to `data/processed/user_dataset_hist23_label7_snapshots_v1.parquet`
- Save split mapping to `data/processed/splits_user_snapshot.csv`
- Save schema report to `artifacts/reports/user_dataset_schema_snapshots_v1.csv`

## Context
- Shared inputs/outputs and execution conventions are documented in the project README.


In [3]:
# ============ Common PATH ============
from pathlib import Path
import numpy as np
import pandas as pd

PROJECT_ROOT = Path(r"C:\Users\seony\Desktop\personal_project\purchase_prediction")

DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"

ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
MODELS_DIR = ARTIFACTS_DIR / "models"
PRED_DIR = ARTIFACTS_DIR / "predictions"
REPORTS_DIR = ARTIFACTS_DIR / "reports"
METRICS_DIR = ARTIFACTS_DIR / "metrics"
FIGURES_DIR = ARTIFACTS_DIR / "figures"

In [4]:
# Inputs / parameters

import numpy as np
import pandas as pd

EVENTS_PATH = PROCESSED_DIR / "events_full_optimized.parquet"

HIST_DAYS = 23
LABEL_DAYS = 7
STEP_DAYS = 7  # snapshot frequency (weekly)

# Split boundaries based on label_end to avoid label leakage across splits
TRAIN_LABEL_END = pd.Timestamp("2021-01-01", tz="UTC")
VALID_LABEL_END = pd.Timestamp("2021-02-01", tz="UTC")

USER_DATASET_OUT = PROCESSED_DIR / "user_dataset_hist23_label7_snapshots_v1.parquet"
SPLITS_OUT = PROCESSED_DIR / "splits_user_snapshot.csv"
SCHEMA_OUT = REPORTS_DIR / "user_dataset_schema_snapshots_v1.csv"

print("EVENTS_PATH:", EVENTS_PATH)
print("USER_DATASET_OUT:", USER_DATASET_OUT)


EVENTS_PATH: C:\Users\seony\Desktop\personal_project\purchase_prediction\data\processed\events_full_optimized.parquet
USER_DATASET_OUT: C:\Users\seony\Desktop\personal_project\purchase_prediction\data\processed\user_dataset_hist23_label7_snapshots_v1.parquet


In [5]:
# Load events

events = pd.read_parquet(EVENTS_PATH)

# Minimal normalization
events["event_type"] = events["event_type"].astype("string").str.lower()
events = events.sort_values("event_time").reset_index(drop=True)

# Hard filter to known event types for safety
events = events[events["event_type"].isin(["view", "cart", "purchase"])].copy()

tmin = events["event_time"].min()
tmax = events["event_time"].max()
print("Events:", events.shape)
print("Time range:", tmin, "->", tmax)
print("Event types:", events["event_type"].value_counts().to_dict())


Events: (885129, 9)
Time range: 2020-09-24 11:57:06+00:00 -> 2021-02-28 23:59:09+00:00
Event types: {'view': 793748, 'cart': 54035, 'purchase': 37346}


In [6]:
# Define cutoffs (weekly snapshots)

# Earliest cutoff must allow full history window
min_cutoff = tmin + pd.Timedelta(days=HIST_DAYS)
max_cutoff = tmax - pd.Timedelta(days=LABEL_DAYS)

cutoffs = pd.date_range(
    start=min_cutoff.normalize() + pd.Timedelta(days=1),
    end=max_cutoff.normalize(),
    freq=f"{STEP_DAYS}D",
    tz="UTC"
)

print("n_cutoffs:", len(cutoffs))
print("cutoff first/last:", cutoffs[0], "->", cutoffs[-1])


n_cutoffs: 19
cutoff first/last: 2020-10-18 00:00:00+00:00 -> 2021-02-21 00:00:00+00:00


In [7]:
# Snapshot builder

def build_snapshot(events: pd.DataFrame, cutoff: pd.Timestamp) -> pd.DataFrame:
    # History and label windows
    hist_start = cutoff - pd.Timedelta(days=HIST_DAYS)
    hist_end = cutoff  # exclusive
    label_start = cutoff
    label_end = cutoff + pd.Timedelta(days=LABEL_DAYS)  # exclusive

    # History slice
    h = events[(events["event_time"] >= hist_start) & (events["event_time"] < hist_end)]

    # Base user feature block
    g = h.groupby("user_id").agg(
        n_events=("event_type", "size"),
        n_sessions=("user_session", "nunique"),
        n_products=("product_id", "nunique"),
        n_categories=("category_id", "nunique"),
        price_mean=("price", "mean"),
        price_max=("price", "max"),
        price_min=("price", "min"),
        last_ts=("event_time", "max"),
    )

    # Event type counts
    type_cnt = h.groupby(["user_id", "event_type"]).size().unstack(fill_value=0)
    for col in ["view", "cart", "purchase"]:
        if col not in type_cnt.columns:
            type_cnt[col] = 0
    type_cnt = type_cnt.rename(columns={
        "view": "n_view",
        "cart": "n_cart",
        "purchase": "n_purchase_hist",
    })

    feat = g.join(type_cnt, how="left")

    # Recency (days)
    feat["recency_days"] = (cutoff - feat["last_ts"]).dt.total_seconds() / 86400.0
    feat = feat.drop(columns=["last_ts"])

    # Simple ratios
    feat["cart_view_ratio"] = feat["n_cart"] / (feat["n_view"] + 1.0)
    feat["purchase_cart_ratio_hist"] = feat["n_purchase_hist"] / (feat["n_cart"] + 1.0)
    feat["events_per_session"] = feat["n_events"] / (feat["n_sessions"] + 1.0)

    # Labels (purchase only in label window)
    l = events[
        (events["event_time"] >= label_start) &
        (events["event_time"] < label_end) &
        (events["event_type"] == "purchase")
    ]

    y = l.groupby("user_id").agg(
        y_purchase=("event_type", "size"),
        y_revenue=("price", "sum"),
    )
    y["y_purchase"] = 1  # at least one purchase in label window

    out = feat.join(y, how="left")

    # Fill label missing -> 0
    out["y_purchase"] = out["y_purchase"].fillna(0).astype("int8")
    out["y_revenue"] = out["y_revenue"].fillna(0.0).astype("float32")

    # Add cutoff metadata
    out = out.reset_index()
    out["cutoff"] = cutoff

    # Assign split by label_end (prevents leakage)
    if label_end <= TRAIN_LABEL_END:
        out["split"] = "train"
    elif label_end <= VALID_LABEL_END:
        out["split"] = "valid"
    else:
        out["split"] = "test"

    return out


In [8]:
# Build dataset (all cutoffs) + quick stats

parts = []
for c in cutoffs:
    snap = build_snapshot(events, c)
    parts.append(snap)

df = pd.concat(parts, axis=0, ignore_index=True)

# Numeric NaN fill 
num_cols = df.select_dtypes(include=["number"]).columns
df[num_cols] = df[num_cols].fillna(0)

print("User snapshot dataset:", df.shape)
print(df[["split", "y_purchase"]].groupby("split").agg(n=("y_purchase","size"), base_rate=("y_purchase","mean")))

# Save split mapping (used later to keep time-based split)
df[["user_id", "cutoff", "split"]].to_csv(SPLITS_OUT, index=False)
print("Saved:", SPLITS_OUT)


User snapshot dataset: (1183222, 19)
            n  base_rate
split                   
test   253026   0.001976
train  651349   0.001442
valid  278847   0.002066
Saved: C:\Users\seony\Desktop\personal_project\purchase_prediction\data\processed\splits_user_snapshot.csv


In [9]:
# Save dataset + schema report

df.to_parquet(USER_DATASET_OUT, index=False)
print("Saved:", USER_DATASET_OUT)

schema = pd.DataFrame({
    "column": df.columns,
    "dtype": [str(df[c].dtype) for c in df.columns],
    "missing_rate": [float(df[c].isna().mean()) for c in df.columns],
})
schema.to_csv(SCHEMA_OUT, index=False)
print("Saved:", SCHEMA_OUT)

display(schema.head(30))

Saved: C:\Users\seony\Desktop\personal_project\purchase_prediction\data\processed\user_dataset_hist23_label7_snapshots_v1.parquet
Saved: C:\Users\seony\Desktop\personal_project\purchase_prediction\artifacts\reports\user_dataset_schema_snapshots_v1.csv


,column,dtype,missing_rate
0,user_id,int64,0.0
1,n_events,int64,0.0
2,n_sessions,int64,0.0
3,n_products,int64,0.0
4,n_categories,int64,0.0
5,price_mean,float32,0.0
6,price_max,float32,0.0
7,price_min,float32,0.0
8,n_cart,int64,0.0
9,n_purchase_hist,int64,0.0


## Data Check

In [10]:
# -----------------------
# 1) Event-level sanity
# -----------------------
print("\n[1] Event types + price sanity")
print(events["event_type"].value_counts(dropna=False).to_string())

price = events["price"].astype(float)
print("\nprice summary:")
print(price.describe(percentiles=[0.01,0.05,0.5,0.95,0.99]).to_string())
print("\nprice <= 0:", int((price <= 0).sum()), "| price isna:", int(price.isna().sum()))

# Purchase rows price sanity
pur = events[events["event_type"] == "purchase"]
print("\n[purchase price summary]")
print(pur["price"].astype(float).describe(percentiles=[0.01,0.05,0.5,0.95,0.99]).to_string())

# Sessions: duplicated sessions across users?
sess_users = events.groupby("user_session")["user_id"].nunique()
print("\n[session uniqueness]")
print("sessions:", sess_users.shape[0])
print("sessions with >1 user_id:", int((sess_users > 1).sum()))

# -----------------------
# 2) Snapshot-level label sanity
# -----------------------
print("\n[2] Split-level label counts")
g = df.groupby("split")["y_purchase"].agg(
    n="size",
    buyers="sum",
    base_rate="mean"
).reset_index()
g["buyers"] = g["buyers"].astype(int)
print(g.to_string(index=False))

# Revenue sanity: revenue should be >0 only when y_purchase==1
bad_rev = df[(df["y_purchase"] == 0) & (df["y_revenue"] > 0)]
print("\nrevenue>0 but y_purchase==0:", len(bad_rev))

rev_buyers = df[df["y_purchase"] == 1]["y_revenue"].astype(float)
print("\n[buyers revenue distribution]")
print(rev_buyers.describe(percentiles=[0.01,0.05,0.5,0.95,0.99]).to_string())

# -----------------------
# 3) Feature distribution checks (detect leakage-like extremes)
# -----------------------
print("\n[3] Key feature summary (overall)")
feat_cols = ["n_events","n_view","n_cart","n_purchase_hist","recency_days","cart_view_ratio","events_per_session"]
print(df[feat_cols].describe(percentiles=[0.01,0.05,0.5,0.95,0.99]).T.to_string())

# Are there users with huge activity that dominate?
print("\nTop 10 n_events snapshots:")
display(df.sort_values("n_events", ascending=False)[["user_id","cutoff","split","n_events","n_view","n_cart","n_purchase_hist","recency_days","y_purchase","y_revenue"]].head(10))

# -----------------------
# 4) Time/split sanity: cutoffs by split + base_rate trend over time
# -----------------------
print("\n[4] Cutoff range by split")
cutoff_rng = df.groupby("split")["cutoff"].agg(["min","max","nunique"]).reset_index()
print(cutoff_rng.to_string(index=False))

print("\nBase rate by cutoff (all splits):")
br = df.groupby(["cutoff"])["y_purchase"].mean().reset_index().sort_values("cutoff")
print(br.tail(10).to_string(index=False))

# -----------------------
# 5) User overlap across splits (expected for snapshot setup)
# -----------------------
print("\n[5] User overlap across splits")
users_train = set(df.loc[df["split"]=="train","user_id"].unique())
users_valid = set(df.loc[df["split"]=="valid","user_id"].unique())
users_test  = set(df.loc[df["split"]=="test","user_id"].unique())

print("n_users_train:", len(users_train))
print("n_users_valid:", len(users_valid))
print("n_users_test :", len(users_test))
print("train∩valid:", len(users_train & users_valid))
print("train∩test :", len(users_train & users_test))
print("valid∩test :", len(users_valid & users_test))

# -----------------------
# 6) Optional: leakage guard (ensure label window not included in features)
# (Quick heuristic: check if any cutoff snapshot has event_time >= cutoff in history feature source)
# This cannot fully prove correctness without rebuilding, but can flag obvious mistakes if present.
# -----------------------
print("\n[6] Quick guard (heuristic)")
print("If you used build_snapshot() from notebook 01, history uses [cutoff-HIST, cutoff) so it should be safe.")


[1] Event types + price sanity
event_type
view        793748
cart         54035
purchase     37346

price summary:
count    885129.000000
mean        146.328713
std         296.807680
min           0.220000
1%            5.750000
5%           10.870000
50%          65.709999
95%         486.809998
99%         889.979980
max       64771.058594

price <= 0: 0 | price isna: 0

[purchase price summary]
count    37346.000000
mean       137.240819
std        169.934693
min          0.220000
1%           1.570000
5%          10.460000
50%         64.480003
95%        479.510010
99%        656.630005
max       3717.649902

[session uniqueness]
sessions: 490398
sessions with >1 user_id: 214

[2] Split-level label counts
split      n  buyers  base_rate
 test 253026     500   0.001976
train 651349     939   0.001442
valid 278847     576   0.002066

revenue>0 but y_purchase==0: 0

[buyers revenue distribution]
count     2015.000000
mean       324.104218
std        537.292250
min          1.570000

,user_id,cutoff,split,n_events,n_view,n_cart,n_purchase_hist,recency_days,y_purchase,y_revenue
941081,1515915625591251010,2021-01-31 00:00:00+00:00,test,363,357,6,0,8.156146,0,0.0
894647,1515915625591251010,2021-01-24 00:00:00+00:00,valid,363,357,6,0,1.156146,0,0.0
999039,1515915625591659523,2021-02-07 00:00:00+00:00,test,280,197,34,49,4.518796,0,0.0
1138540,1515915625599852988,2021-02-21 00:00:00+00:00,test,265,265,0,0,0.902037,0,0.0
993049,1515915625598794428,2021-01-31 00:00:00+00:00,test,260,260,0,0,0.122095,0,0.0
1084836,1515915625598794428,2021-02-14 00:00:00+00:00,test,260,260,0,0,14.122095,0,0.0
1131075,1515915625598794428,2021-02-21 00:00:00+00:00,test,260,260,0,0,21.122095,0,0.0
1038869,1515915625598794428,2021-02-07 00:00:00+00:00,test,260,260,0,0,7.122095,0,0.0
1061948,1515915625591659523,2021-02-14 00:00:00+00:00,test,256,178,31,47,11.518796,0,0.0
1121736,1515915625536567608,2021-02-21 00:00:00+00:00,test,256,180,57,19,1.259016,0,0.0



[4] Cutoff range by split
split                       min                       max  nunique
 test 2021-01-31 00:00:00+00:00 2021-02-21 00:00:00+00:00        4
train 2020-10-18 00:00:00+00:00 2020-12-20 00:00:00+00:00       10
valid 2020-12-27 00:00:00+00:00 2021-01-24 00:00:00+00:00        5

Base rate by cutoff (all splits):
                   cutoff  y_purchase
2020-12-20 00:00:00+00:00    0.002111
2020-12-27 00:00:00+00:00    0.001487
2021-01-03 00:00:00+00:00    0.001552
2021-01-10 00:00:00+00:00    0.002342
2021-01-17 00:00:00+00:00    0.002471
2021-01-24 00:00:00+00:00    0.002428
2021-01-31 00:00:00+00:00    0.002048
2021-02-07 00:00:00+00:00    0.001917
2021-02-14 00:00:00+00:00    0.001945
2021-02-21 00:00:00+00:00    0.001995

[5] User overlap across splits
n_users_train: 230414
n_users_valid: 124072
n_users_test : 119096
train∩valid: 39484
train∩test : 2422
valid∩test : 46072

[6] Quick guard (heuristic)
If you used build_snapshot() from notebook 01, history uses [cutoff-H

In [11]:
USER_DATASET_OUT = PROCESSED_DIR / "user_dataset_hist23_label7_snapshots_v1.parquet"
df = pd.read_parquet(USER_DATASET_OUT)

# -----------------------
# 1) Snapshots per user distribution
# -----------------------
u = df.groupby("user_id").size().rename("n_snapshots").reset_index()
print("[1] n_snapshots per user (distribution)")
print(u["n_snapshots"].describe(percentiles=[0.01,0.05,0.5,0.95,0.99]).to_string())
print("\nTop 10 users by n_snapshots:")
display(u.sort_values("n_snapshots", ascending=False).head(10))

# -----------------------
# 2) Buyers vs non-buyers feature contrast (signal check)
# -----------------------
key_feats = ["n_events","n_view","n_cart","n_purchase_hist","recency_days","cart_view_ratio","events_per_session","price_mean","price_max"]
have = [c for c in key_feats if c in df.columns]

def summarize(group_df: pd.DataFrame, label: str) -> pd.DataFrame:
    x = group_df[have].astype(float)
    out = pd.DataFrame({
        "group": label,
        "feature": have,
        "mean": x.mean().values,
        "median": x.median().values,
        "p95": x.quantile(0.95).values,
        "p99": x.quantile(0.99).values,
    })
    return out

buyers = df[df["y_purchase"] == 1]
nonbuyers = df[df["y_purchase"] == 0]

sum_buy = summarize(buyers, "buyers")
sum_non = summarize(nonbuyers, "non_buyers")
cmp = sum_buy.merge(sum_non, on="feature", suffixes=("_buyers","_non"))
cmp["mean_ratio_buy/non"] = cmp["mean_buyers"] / (cmp["mean_non"] + 1e-9)
cmp = cmp.sort_values("mean_ratio_buy/non", ascending=False)

print("\n[2] Buyers vs non-buyers (feature contrast)")
display(cmp)

# -----------------------
# 3) Simple uplift by deciles for one strong signal (n_cart) as a quick proxy
# -----------------------
tmp = df[["y_purchase","n_cart"]].copy()
tmp["n_cart"] = tmp["n_cart"].astype(float)
tmp["bin"] = pd.qcut(tmp["n_cart"].rank(method="first"), 10, labels=False) + 1  # 1..10
dec = tmp.groupby("bin").agg(
    n=("y_purchase","size"),
    base_rate=("y_purchase","mean"),
    avg_n_cart=("n_cart","mean")
).reset_index().sort_values("bin")

print("\n[3] Base rate by n_cart decile (1=low, 10=high)")
print(dec.to_string(index=False))

[1] n_snapshots per user (distribution)
count    387330.000000
mean          3.054816
std           1.021746
min           1.000000
1%            1.000000
5%            1.000000
50%           3.000000
95%           4.000000
99%           6.000000
max          19.000000

Top 10 users by n_snapshots:


,user_id,n_snapshots
21094,1515915625521534659,19
4111,1515915625494574825,19
1986,1515915625427007765,19
36164,1515915625523780315,19
125,1515915625355398801,19
23,1515915625353487799,19
35978,1515915625523753462,19
4618,1515915625505475638,19
131,1515915625355611972,19
49867,1515915625526052886,19



[2] Buyers vs non-buyers (feature contrast)


,group_buyers,feature,mean_buyers,median_buyers,p95_buyers,p99_buyers,group_non,mean_non,median_non,p95_non,p99_non,mean_ratio_buy/non
3,buyers,n_purchase_hist,0.975186,0.000000,4.000000,8.000000,non_buyers,0.085819,0.000000,1.000000,2.000000,11.363289
2,buyers,n_cart,1.332506,1.000000,4.000000,12.860000,non_buyers,0.123835,0.000000,1.000000,2.000000,10.760319
5,buyers,cart_view_ratio,0.168395,0.111111,0.500000,0.666667,non_buyers,0.029480,0.000000,0.333333,0.500000,5.712143
0,buyers,n_events,9.860050,5.000000,33.300000,88.860000,non_buyers,2.075913,1.000000,6.000000,14.000000,4.749741
1,buyers,n_view,7.552357,3.000000,27.300000,70.860000,non_buyers,1.866259,1.000000,5.000000,12.000000,4.046790
6,buyers,events_per_session,2.894672,1.666667,8.680952,21.480000,non_buyers,0.895356,0.500000,2.500000,5.000000,3.232986
8,buyers,price_max,230.278630,133.029999,674.741998,1274.544016,non_buyers,138.772549,56.889999,486.809998,1031.589966,1.659396
7,buyers,price_mean,174.269945,113.809998,505.391986,836.247197,non_buyers,129.064957,55.023331,466.519989,935.320007,1.350250
4,buyers,recency_days,4.858012,2.536389,18.173659,21.599034,non_buyers,11.268202,11.275093,21.653751,22.653784,0.431126



[3] Base rate by n_cart decile (1=low, 10=high)
 bin      n  base_rate  avg_n_cart
   1 118323   0.000651    0.000000
   2 118322   0.000685    0.000000
   3 118322   0.000541    0.000000
   4 118322   0.000583    0.000000
   5 118322   0.000659    0.000000
   6 118322   0.000980    0.000000
   7 118322   0.000972    0.000000
   8 118322   0.000972    0.000000
   9 118322   0.000862    0.000000
  10 118323   0.010125    1.258927
